In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.sample(8)

,age,gender,fever,cough,city,has_covid
9,64,Female,101.0,Mild,Delhi,No
61,81,Female,98.0,Strong,Mumbai,No
93,27,Male,100.0,Mild,Kolkata,Yes
89,46,Male,103.0,Strong,Bangalore,No
79,48,Female,103.0,Mild,Kolkata,Yes
50,19,Male,101.0,Mild,Delhi,Yes
87,47,Male,101.0,Strong,Bangalore,No
18,64,Female,98.0,Mild,Bangalore,Yes


In [5]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [6]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [7]:
#Age = Numerical Data(no preprocessing)
#Gender = Nominal Categorical Data(OHE)
# Fever = Numerical(Simple IMputer)
#cough = Ordinal data(Ordinal Encoder)
#city = Nominal Data(OHE)

In [8]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

In [12]:
X = df.iloc[:,0:5]
Y = df['has_covid']

In [14]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)

In [24]:
x_train[['fever']]

,fever
1,100.0
42,100.0
83,104.0
44,102.0
16,103.0
...,...
86,104.0
63,100.0
88,100.0
4,101.0


## The Hard way

In [19]:
#Adding simple Imputer 
si=SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])
#on test data too
x_test_fever = si.fit_transform(x_test[['fever']])

#some questioo occured need to ask if u see this again!! remember i am from past..

In [25]:
x_train_fever.shape

(80, 1)

In [27]:
#Ordinal encoding in ->cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])

In [28]:
oe.fit(x_train[['cough']])

OrdinalEncoder(categories=[['Mild', 'Strong']])

In [30]:
x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.fit_transform(x_test[['cough']])

x_train_cough.shape

In [35]:
#On Hot Encoding ->Gender,City
ohe = OneHotEncoder(drop='first',sparse=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

C:\Users\HP\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\HP\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [37]:
x_train_gender_city.shape

(80, 4)

In [46]:
#We need Age
x_train_Age = x_train[['age']].values
x_test_Age  = x_test[['age']].values
x_train_Age.shape

(80, 1)

## Now we concatenate all data we made

In [50]:
x_train_transformed = np.concatenate((x_train_Age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
x_test_transformed = np.concatenate((x_test_Age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

In [52]:
x_train_transformed.shape

(80, 7)

## The Easy Way[ Column Transformer ] 

In [53]:
from sklearn.compose import ColumnTransformer

In [54]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [59]:
transformer.transform(x_train).shape

(80, 7)

In [60]:
transformer.transform(x_test).shape

(20, 7)